<a href="https://colab.research.google.com/github/michaelgfalk/clean-ocr/blob/master/ocr_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waves of Words: Correcting Trove's Messy OCR

One aim of the *Waves of Words* project is to extract Aboriginal wordlists from [Trove](https://trove.nla.gov.au). A challenge we face is that historical newspapers are difficult to OCR, so many of the texts are riddled with errors.

Using the training data available from the ALTA 2017 OCR competition, can we create a model that will clean the text enough for our aboriginal word detector to work?

I have been giving some thought to whether uppercase letters and punctuation should be preserved in this model, given that the aim is to clean up the text for our detector, which only requires lower case letters and ignores punctuation. I think we need to include all the characters in this one. The extra information about sentence barriers, for example, should hopefully help the model as it would a human when it tries to correct the text. Moreover, many OCR errors involve exchaning punctuation or digits for letters, e.g. `l = 1 = !`.

**References:**

* D. Mollá, S. Cassidy. Overview of the 2017 ALTA Shared Task:
Correcting OCR Errors (2017). *Proc. ALTA 2017*.
[https://aclanthology.coli.uni-saarland.de/papers/U17-1014/u17-1014](https://aclanthology.coli.uni-saarland.de/papers/U17-1014/u17-1014)

In [0]:
# Install TensorFlow2
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [0]:
from __future__ import absolute_import, division, print_function

from google.colab import drive

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import csv
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split

In [3]:
# Mount google drive to get training data. Set data_dir
drive.mount('/content/gdrive')
data_dir = '/content/gdrive/My Drive/waves_of_words/ocr_correction_data/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 1. Data Pipeline (new)

This new data pipeline makes use of Tensorflows Dataset object to manage the extraction and transformation more efficiently.

In [0]:
def create_dataset(source, target, start = 'स', end = 'ए'):
  """Imports articles from csv and packages for training."""
  
  # Lists for import
  raw_x = []
  raw_y = []
  
  # Import raw text
  with open(source, "rt") as f:
    x_reader = csv.reader(f, delimiter = ',', quotechar = '"')
    for row in x_reader:
      raw_x.append(row[1])
  with open(target, "rt") as f:
    y_reader = csv.reader(f, delimiter = ',', quotechar = '"')
    for row in y_reader:
      raw_y.append(row[1])
  
  # Drop header rows
  raw_x = raw_x[1:]
  raw_y = raw_y[1:]
  
  # Add special characters
  x = [start + article + end for article in raw_x if start not in article and end not in article]
  y = [start + article + end for article in raw_y if start not in article and end not in article]
  
  return x, y

In [0]:
def tokenize(source, target):
  """Instantiates tokenizer, fits and applies."""
  
  # Fit tokenizer
  tkzr = tf.keras.preprocessing.text.Tokenizer(
    num_words = None,
    filters = None,
    lower = False,
    char_level = True
  )
  tkzr.fit_on_texts(source + target)
  
  # Apply to texts
  x = tkzr.texts_to_sequences(source)
  y = tkzr.texts_to_sequences(target)
  
  return x, y, tkzr

In [0]:
def split_and_stack(x, y, max_len, batch_size, drop = False):
  """Takes as input two python lists, and outputs a list of tensor buckets.
  
  Arguments:
  ==========
  x (list): the tokenized source strings
  y (list): the tokenized target strings
  max_len (int): the maximum number of time steps the model will consider
  batch_size (int): the batch size for the training examples
  drop (bool): keep the final batch, if len(x) is not a multiple of batch_size?
  
  Returns:
  ==========
  bucket_list (list): a list of length m, each item of which is a bucket of
    similar-length numpy array
  
  A bucket is a tensor of shape (2, m_prime, max_len). Training examples are first
  bucketed into groups of similar length, and seperated into batches of batch_size.
  Each batch is then padded out to an integer multiple of max_len, split into chunks
  of length max_len and stacked.
  
  This bucketing, splitting and stacking allows the data to be fed to a stateful RNN.
  
  Dimensions:
  -The first dimension seperates x examples from y
  -The second dimension seperates individual training examples.
    m_prime = batch_size * ⌈max_len_x_or_y_within_batch / batch_size⌉ 
  -The third dimension seperates individual time-steps, and is fixed at max_len"""
  
  # Set number of training examples (round up if not dropping)
  assert len(x) == len(y)
  m = len(x)
  
  # Sort x and y by sequence length
  x_y = sorted(zip(x,y), key = lambda tup: max(len(tup[0]), len(tup[1])), reverse = True)
  
  # Loop through list and create batches
  bucket_list = []
  for i in range(0, m, batch_size):
    
    # Slice and unpack the list
    bn = batch_size
    if len(x_y) < batch_size and drop:
      break
    elif len(x_y) < batch_size and not drop:
      bn = len(x_y)
    
    bx, by = zip(*[x_y.pop() for _ in range(bn)])
    
    # Calculate length boundary and m_prime
    bl = max(len(bx[0]), len(by[0]))
    b = max_len - (bl % max_len) + bl
    m_prime = int(b / max_len)
    
    # Pad the sequences
    x_pad = pad_sequences(bx, maxlen = b, padding = 'post')
    y_pad = pad_sequences(by, maxlen = b, padding = 'post')
    
    # Split and stack
    x_out = np.concatenate(np.split(x_pad, m_prime, axis = 1), axis = 0)
    y_out = np.concatenate(np.split(y_pad, m_prime, axis = 1), axis = 0)
    
    # Covert to dataset and append to list
    bucket = tf.data.Dataset.from_tensor_slices((x_out, y_out))
    bucket = bucket.batch(bn) # NB: bn = batch_size except on the last batch, if drop != True
    bucket_list.append(bucket)
    
  return bucket_list

In [0]:
def load_dataset(x_path, y_path, max_len = 20, batch_size = 128, drop = False, test_size = .2):
  
  x, y = create_dataset(x_path, y_path)
  
  x, y, tkzr = tokenize(x, y)
  
  seqs = (x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size = test_size)
  
  train_buckets = split_and_stack(x_train, y_train, max_len, batch_size, drop)
  
  test_buckets = split_and_stack(x_test, y_test, max_len, batch_size, drop)
  
  return train_buckets, test_buckets, tkzr, seqs

# 2. Model Definition

Adapted from TensorFlow docs.

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, num_chars, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(num_chars, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, 
                                   # The following parameters must be set this way
                                   # to use CuDNN on GPU
                                   activation='tanh',
                                   recurrent_activation='sigmoid',
                                   recurrent_dropout=0,
                                   unroll=False,
                                   use_bias=True,
                                   reset_after=True,
                                   # The following parameters are necessary for the
                                   # encoder-decoder architecture
                                   return_sequences=True, 
                                   return_state=True,
                                   # Stateful must be 'True' in order
                                   # to link the batches in each hyperbatch
                                   stateful=True,
                                   # Just the standard initializer
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)        
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  
  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, hidden_size)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    
    return context_vector, attention_weights

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, num_chars, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(num_chars, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, 
                                   # The following parameters must be set this way
                                   # to use CuDNN on GPU
                                   activation='tanh',
                                   recurrent_activation='sigmoid',
                                   recurrent_dropout=0,
                                   unroll=False,
                                   use_bias=True,
                                   reset_after=True,
                                   # The following parameters are necessary for the
                                   # encoder-decoder architecture
                                   return_sequences=True, 
                                   return_state=True,
                                   # Stateful must be 'True' in order
                                   # to link the batches in each hyperbatch
                                   stateful=True,
                                   # Just the standard initializer
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(num_chars)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

# 3. Set up Training

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
  # Mask: ignore the model's predictions where the ground truth is padding
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  
  # Calculate the loss
  loss_ = loss_object(real, pred)

  # Make mask compatible with the loss output
  mask = tf.cast(mask, dtype=loss_.dtype)
  
  # Multiply the losses by the mask (i.e. zero out all losses where there's just padding)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden, norm_lim):
  loss = 0
        
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims(inp[:,0], 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)
  
  # Clip gradients
  clipped_gradients = [tf.clip_by_norm(grad, norm_lim) for grad in gradients]

  optimizer.apply_gradients(zip(clipped_gradients, variables))
  
  return batch_loss

In [0]:
@tf.function
def val_step(inp, targ, enc_hidden):
  
  loss = 0
  
  # Begin feeding data to network
  enc_output, enc_hidden = encoder(inp, enc_hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims(inp[:,0], 1)
  
  # Cycle through the rest of the time steps
  for t in range(1, targ.shape[1]):
    # Pass enc_output to the decoder
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
    loss += loss_function(targ[:,t], predictions)
    # Pass the next correct letter to the decoder (teacher forcing)
    dec_input = tf.expand_dims(targ[:,t], 1)
    
  # Calculate val_loss
  val_loss = (loss / int(targ.shape[1]))
  
  return val_loss

In [0]:
def format_time(flt):
  h = flt//3600
  m = (flt % 3600)//60
  s = flt % 60
  out = []
  if h > 0:
    out.append(str(int(h)))
    if h == 1:
      out.append('hr,')
    else:
      out.append('hrs,')
  if m > 0:
    out.append(str(int(m)))
    if m == 1:
      out.append('min, and')
    else:
      out.append('mins, and')
  out.append(f'{s:.2f}')
  out.append('secs')
  return ' '.join(out)

# 4. Run Training Loop

In [0]:
# Set hyperparameters
MAX_LEN = 32
BATCH_SIZE = 128
EPOCHS = 2
x_dir = data_dir + 'train_input.csv'
y_dir = data_dir + 'train_output.csv'
NORM_LIM = 3 # value for clip_norm

# Load data
train_buckets, test_buckets, tkzr, seqs = load_dataset(x_dir, y_dir, MAX_LEN, BATCH_SIZE, drop = True)

# Get vocab size
num_chars = len(tkzr.word_index) + 1 # Add one for padding
embedding_dim = 15
enc_hidden = 30
dec_hidden = 30

# Define model(s)
encoder = Encoder(num_chars, embedding_dim, enc_hidden, batch_sz = BATCH_SIZE)
decoder = Decoder(num_chars, embedding_dim, dec_hidden, batch_sz = BATCH_SIZE)

In [0]:
checkpoint_dir = data_dir + 'checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [33]:
# Loop over epochs
for epoch in range(EPOCHS):
  print(f'Starting Epoch {epoch + 1}\n')
  
  start = time.time()
  
  total_loss = 0
  total_batches = 0
  val_loss = 0
  val_batches = 0

  # Loop over buckets
  for bucket, dataset in enumerate(train_buckets):
    # Reset hidden state
    enc_hidden = encoder.initialize_hidden_state()


    for inp, targ in dataset.take(-1):
      batch_loss = train_step(inp, targ, enc_hidden, NORM_LIM)
      
      if batch_loss.numpy() == np.nan:
        print(f'NaN loss! E = {epoch+1}, B = {bucket + 1}, b = {total_batches}')
        checkpoint.save(file_prefix = checkpoint_prefix)
        last_batch = (inp, targ)
        
      total_loss += batch_loss
      
      total_batches += 1
    
      if total_batches % 500 == 0:
          print(f'Epoch {epoch + 1} Bucket {bucket + 1} Loss {batch_loss.numpy():.4f} after {total_batches} batches')
  
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    
  # Calculate validation loss
  for dataset in test_buckets:
    # Reset hidden state
    enc_hidden = encoder.initialize_hidden_state()
    
    for inp, targ in dataset.take(-1):
      batch_val_loss = val_step(inp, targ, enc_hidden)
      
      val_loss += batch_val_loss
      val_batches += 1

  print(f'\nEpoch {epoch + 1} Loss {(total_loss / total_batches):.4f} after {total_batches} batches.')
  print(f'Tested on {val_batches * BATCH_SIZE} validation examples. val_loss = {val_loss / val_batches}')
  print(f'Time taken for 1 epoch: {format_time(time.time() - start)}\n===========================\n\n')

Starting Epoch 1

Epoch 1 Bucket 21 Loss 1.0739 after 500 batches
Epoch 1 Bucket 28 Loss 0.8486 after 1000 batches
Epoch 1 Bucket 32 Loss 0.9278 after 1500 batches
Epoch 1 Bucket 35 Loss 1.1902 after 2000 batches
Epoch 1 Bucket 37 Loss 1.5162 after 2500 batches

Epoch 1 Loss 1.1684 after 2645 batches.
Tested on 62336 validation examples. val_loss = 1.058976173400879
Time taken for 1 epoch: 9 mins, and 57.31 seconds.


Starting Epoch 2

Epoch 2 Bucket 21 Loss 1.0332 after 500 batches
Epoch 2 Bucket 28 Loss 0.8051 after 1000 batches
Epoch 2 Bucket 32 Loss 0.8842 after 1500 batches
Epoch 2 Bucket 35 Loss 1.1504 after 2000 batches
Epoch 2 Bucket 37 Loss 1.4829 after 2500 batches

Epoch 2 Loss 1.1292 after 2645 batches.
Tested on 62336 validation examples. val_loss = 1.0281800031661987
Time taken for 1 epoch: 10 mins, and 5.55 seconds.


Starting Epoch 3

Epoch 3 Bucket 21 Loss 0.9999 after 500 batches
Epoch 3 Bucket 28 Loss 0.7834 after 1000 batches
Epoch 3 Bucket 32 Loss 0.8613 after 1500

KeyboardInterrupt: ignored

# See How it Goes

In [0]:
def clean_ocr(article, tkzr, start_char = 'स', end_char = 'ए'):

    # Tokenise the article
    inputs = [inp_lang.word_index[i] for i in list(article)]
    
    # Set threshold to stop outputting results if 'end' character not reached
    give_up = int(len(inputs) * 1.2)
    
    # Convert to tensor
    inputs = tf.convert_to_tensor(inputs)
    
    # Create empty string for result
    result = ''
    
    # Encode the article
    hidden = np.zeroes(1, encoder.enc_units)
    enc_out, enc_hidden = encoder(inputs, hidden)

    # Ready the decoder
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tkzr.word_index[start_char]], 0)

    # Start outputting characters till the end character is reached, or
    # the output sequence is getting much longer than the input
    while len(result) < give_up:
      # Get next result from decoder:
      predictions, dec_hidden, _ = decoder(dec_input,
                                           dec_hidden,
                                           enc_out)
      
      # Which is the next letter?
      predicted_id = tf.argmax(predictions[0]).numpy()
      
      # Add to result
      result += tkzr.index_word[predicted_id]

      # If we've reached the end, stop
      if tkzr.index_word[predicted_id] == end_char:
          break

      # Otherwise the predicted ID is fed back into the model
      dec_input = tf.expand_dims([predicted_id], 0)

    return result, article